In [1]:
import numpy as np
import pandas as pd
from pmdarima import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection_sarimax import backtesting_sarimax
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('once')

# Carga de datos

In [2]:
df_raw = pd.read_csv("datos_preprocesados.csv", parse_dates=['fecha'], index_col='fecha').asfreq('D')
df = df_raw.copy()

# Estacionareidad

In [3]:
# Realizar la prueba ADF
result = adfuller(df['demanda'].dropna(), autolag='AIC')  # Reemplaza 'demanda' con el nombre de tu columna

# Mostrar los resultados
print('Estadístico ADF:', result[0])
print('Valor p:', result[1])
print('Número de lags utilizados:', result[2])
print('Número de observaciones:', result[3])
print('Valores críticos:')
for key, value in result[4].items():
    print(f'   {key}: {value:.3f}')

Estadístico ADF: -5.044953538402203
Valor p: 1.8068747033700663e-05
Número de lags utilizados: 26
Número de observaciones: 2061
Valores críticos:
   1%: -3.434
   5%: -2.863
   10%: -2.568


# División del conjunto de datos

In [4]:
# Split train-validation-test
# ==============================================================================
end_train = '2021-12-31'
end_validation = '2022-12-31'
data_train = df.loc[: end_train, :]
data_val   = df.loc[end_train:end_validation, :]
data_test  = df.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validacion : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Dates train      : 2019-01-01 00:00:00 --- 2021-12-31 00:00:00  (n=1096)
Dates validacion : 2021-12-31 00:00:00 --- 2022-12-31 00:00:00  (n=366)
Dates test       : 2022-12-31 00:00:00 --- 2024-09-18 00:00:00  (n=628)


# Modelo ARIMAX

In [5]:
def arimax(exog_features, p, d, q, maxiter):
    forecaster = ForecasterSarimax( regressor=ARIMA(order=(p, d, q), seasonal_order=(0, 0, 0, 0), maxiter=maxiter))
    
    metric, predictions = backtesting_sarimax(
        forecaster         = forecaster,
        y                  = df['demanda'],
        exog               = df[exog_features],
        steps              = 1,
        metric             = 'mean_absolute_error',
        initial_train_size = len(df[:end_validation]),
        refit              = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = True
)
    return metric
             

# Resultados

In [6]:
exog_features = ['tmed', 'hrmedia', 'trim', 'diasem', 'festivo']
maxiter = 10

# Definir los rangos para p, d y q
range_values = range(1,2)  # Esto generará 0, 1, 2

# Crear una lista para almacenar los resultados
results = []

# Iterar sobre los valores de p, d, q
for p in range_values:
    for d in range_values:
        for q in range_values:
            # Llamar a la función arimax con los parámetros actuales
            metric = arimax(exog_features, p, d, q, maxiter)
            # Imprimir el resultado
            print(f"Para p, d, q = {p}, {d}, {q} la métrica es {metric:.4f}")
            # Almacenar los resultados
            results.append((p, d, q, metric))

# Convertir la lista de resultados a un DataFrame
results_df = pd.DataFrame(results, columns=['p', 'd', 'q', 'metric'])

# Guardar los resultados en un archivo CSV
results_df.to_csv('resultados_arimax_3.csv', index=False)

print("Los resultados se han guardado en 'resultados_arimax.csv'.")


C:\Users\grego\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\grego\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\grego\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


  0%|          | 0/627 [00:00<?, ?it/s]

Para p, d, q = 1, 1, 1 la métrica es 1140.5867
Los resultados se han guardado en 'resultados_arimax.csv'.


# Representación gráfica

In [ ]:
# Plot predictions vs real value
# ======================================================================================
plt.figure(figsize=(10, 5))  # Define el tamaño de la figura

# Graficar los valores reales
plt.plot(data_test.index, data_test['demanda'], label="Real Value", color='blue', linestyle='-')

# Graficar las predicciones
plt.plot(predictions.index, predictions['pred'], label="Predictions", color='orange', linestyle='--')

# Configurar el título y las etiquetas
plt.title("Real Value vs Predicted in Test Data")
plt.xlabel("Date Time")
plt.ylabel("Users")

# Añadir leyenda
plt.legend(loc='upper left')

# Mostrar el gráfico
plt.grid()  # Añadir cuadrícula para mejor visualización
plt.tight_layout()  # Ajustar el layout
plt.show()
